In [ ]:
[{"id":"28c4cf80-e53e-11e8-a6c4-1391f936ee56","kind":"ADULT_FEMALE","top":286,"left":392,"bottom":323,"right":425},{"id":"28c4f690-e53e-11e8-a6c4-1391f936ee56","kind":"ADULT_FEMALE","top":115,"left":180,"bottom":150,"right":227}]

In [ ]:
import glob
from keras.applications.resnet50 import ResNet50
from keras import layers
from keras.models import Model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## data

In [ ]:
data = glob.glob("/root/data/lice/lice_detection/0_lice_classification/*/*")

In [ ]:
classdict = {}
for d in data:
    liceclass = d.split('/')[-2]
    if liceclass == "static" or liceclass == "moving":
        liceclass = "static_moving"
    if liceclass not in classdict:
        classdict[liceclass] = []
    classdict[liceclass].append(d)

## model

In [ ]:
BATCH_SIZE = 16
INPUT_SHAPE = (128, 128, 3)

In [ ]:
resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

In [ ]:
x = layers.GlobalAveragePooling2D(name='avg_pool')(resnet50.output)
x = layers.Dense(3, activation='softmax', name='fc1000')(x)

In [ ]:
model = Model(inputs=[resnet50.input], outputs=[x])

In [ ]:
# adam = Adam(lr=1e-3)
model.compile("adam", loss="categorical_crossentropy", metrics=['categorical_accuracy'])

## generator

In [ ]:
from albumentations import PadIfNeeded
import skimage.io as io
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
aug = PadIfNeeded(p=1, min_height=INPUT_SHAPE[0], min_width=INPUT_SHAPE[1], border_mode=0)

In [ ]:
CLASS_MAP = {"adult_female": 2, 
             "static_moving": 1,
             "uncertain":0}
CLASS_MAP2 = {0: "not_lice", 1: "moving/fixed", 2: "adult female"}

In [ ]:
classdict.keys()

In [ ]:
def myGenerator(classdict, BATCH_SIZE, INPUT_SHAPE):
    i = 0

    while True:
        x_batch = np.zeros((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, 3))
        for i in range(BATCH_SIZE):
            liceclass = np.random.choice(list(classdict.keys()))
            img_path = np.random.choice(classdict[liceclass])
            image = io.imread(img_path)
            augmented = aug(image=image)
            x = cv2.resize(augmented["image"], (INPUT_SHAPE[0], INPUT_SHAPE[1]), interpolation=cv2.INTER_LINEAR)
            y = np.zeros((3))
            y[CLASS_MAP[liceclass]] = 1
            x_batch[i, ...] = x
            y_batch[i, ...] = y
        yield x_batch, y_batch

In [ ]:
steps_per_epoch = len(data) // BATCH_SIZE

In [ ]:
# create the generators
train_generator = myGenerator(classdict, BATCH_SIZE, INPUT_SHAPE)

In [ ]:
xb, yb = next(train_generator)

In [ ]:
for i in range(BATCH_SIZE):
    plt.imshow(xb[i, ...])
    plt.title(CLASS_MAP2[int(np.argmax(yb[i, ...]))])
    plt.show()

## Train

In [ ]:
from datetime import datetime

In [ ]:
datetime.now().strftime("%Y%m%d_%HH%M")

In [ ]:
!nvidia-smi

In [ ]:
# start training
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        verbose=1)

In [ ]:
model.save("/root/data/models/lice/lice_detection_cv_algorithms.h5")

In [ ]:
np.argmin(history.history["loss"])